## Phase 9 : Risque de Transition et Stress Test ECL
**Le But de l'Étude (Contexte Métier)**

L'objectif de cette phase est de répondre à la question suivante : Quel est l'impact financier futur sur nos pertes attendues ***(ECL)*** si la transition vers une économie bas-carbone se déroule mal ou trop vite ***(stress de Transition)*** ?

Contrairement au Risque Physique (qui dépend de la géographie), le Risque de Transition dépend de ***la politique, de la technologie, et du marché (ex: taxe carbone, coût de l'énergie)*** qui affectent directement la capacité de remboursement des emprunteurs.

In [7]:
# Imports nécessaires
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression



# 1.1 Charger les données de base (X et y)
X = pd.read_csv("../data/processed/X_features.csv")
y = pd.read_csv("../data/processed/y_target.csv").squeeze()

# 1.2. Séparer les données en Training/Test pour obtenir le bon Scaler et le bon Model
# (Réplication de la logique de training de la Phase 5/8 pour la cohérence)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 1.3. Scaling et Entraînement du Modèle Baseline
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

model = LogisticRegression(
    penalty="l2",
    C=1.0,
    solver="lbfgs",
    max_iter=1000,
    random_state=42
)
model.fit(X_train_scaled, y_train)

# 1.4. Calcul de la PD Baseline (sur tout le dataset X)
X_scaled_all = scaler.transform(X)
pd_baseline = model.predict_proba(X_scaled_all)[:, 1]

# 1.5. Définition des Proxies ECL (tirés du glossaire et de la Phase 8)
LGD = 0.60  # Loss Given Default (60% de perte en cas de défaut)
EAD = X["loan_amnt"] # Exposure at Default (Montant du prêt)

# 1.6. Calcul de l'ECL Baseline (en euros)
ECL_baseline = pd_baseline * LGD * EAD

### Définition des Scénarios de Transition

Nous définissons les scénarios(Baseline, Ordonnée, Désordonnée, Monde Chaud). Nous traduisons l'impact de ces scénarios par un choc sur la variable annual_inc_clean (revenu annuel net), qui est un facteur clé du risque.

In [3]:
# 2. Définition des Scénarios de Choc sur la variable 'annual_inc_clean'

# Taux de choc appliqués à l'Annual Income pour simuler la vulnérabilité macro
CHOCS_TRANSITION = {
    "baseline": 0.00,
    "orderly": -0.03,        # Réduction modérée (-3%)
    "disorderly": -0.10,     # Réduction forte (-10%)
    "hothouse": -0.15        # Réduction très forte (-15%)
}

# DataFrame pour stocker les résultats ECL par scénario
results_ecl = []

In [15]:
# 3. Simulation et Calcul de la PD stressée et de l'ECL

for scenario, shock_rate in CHOCS_TRANSITION.items():
    print(f"--- Simulation du scénario: {scenario} (Choc: {shock_rate*100:.0f}%) ---")
    
    # 3.1. Cloner le jeu de features pour l'application du choc
    X_stressed = X.copy()
    
    # 3.2. Appliquer le choc sur la variable 'annual_inc_clean'
    # Le choc est appliqué sur les valeurs réelles non scalées (X)
    
    # On assure que le revenu ne devienne pas négatif
    X_stressed["annual_inc_clean"] = X_stressed["annual_inc_clean"].apply(
        lambda x: max(0, x * (1 + shock_rate)) 
    )

    # 3.3. Scaler les features stressées (en utilisant le scaler ajusté sur le training set)
    X_stressed_scaled = scaler.transform(X_stressed)
    
    # 3.4. Prédire la PD sous stress
    pd_stressed = model.predict_proba(X_stressed_scaled)[:, 1]
    
    # 3.5. Calculer l'ECL stressée
    ecl_stressed = pd_stressed * LGD * EAD
    
    # 3.6. Enregistrement des résultats agrégés
    results_ecl.append({
        "scenario": scenario,
        "ECL_Total_MM_USD": ecl_stressed.sum() / 1_000_000,
        "PD_Mean": pd_stressed.mean()
    })


# Convertir les résultats en DataFrame
df_stress_test = pd.DataFrame(results_ecl)

df_stress_test

--- Simulation du scénario: baseline (Choc: 0%) ---
--- Simulation du scénario: orderly (Choc: -3%) ---
--- Simulation du scénario: disorderly (Choc: -10%) ---
--- Simulation du scénario: hothouse (Choc: -15%) ---


,scenario,ECL_Total_MM_USD,PD_Mean
0,baseline,467.179365,0.199642
1,orderly,470.774752,0.201009
2,disorderly,479.264489,0.204234
3,hothouse,485.414937,0.206568
4,baseline,467.179365,0.199642
5,orderly,470.774752,0.201009
6,disorderly,479.264489,0.204234
7,hothouse,485.414937,0.206568
8,baseline,467.179365,0.199642
9,orderly,470.774752,0.201009


### Analyse et Export du Delta ECL

Nous comparons l'ECL de chaque scénario à la Baseline pour quantifier l'impact financier de la transition.

In [10]:
# 4. Analyse des Résultats et Calcul du Delta ECL

# 4.1. Récupérer la valeur ECL de la Baseline
ecl_baseline_value = df_stress_test[
    df_stress_test["scenario"] == "baseline"
]["ECL_Total_MM_USD"].iloc[0]

# 4.2. Calcul du Delta ECL (la perte supplémentaire en millions USD)
df_stress_test["Delta_ECL_MM_USD"] = (
    df_stress_test["ECL_Total_MM_USD"] - ecl_baseline_value
)

# 4.3. Mise en forme et tri pour l'export
df_stress_test["ECL_Total_MM_USD"] = df_stress_test["ECL_Total_MM_USD"].round(2)
df_stress_test["Delta_ECL_MM_USD"] = df_stress_test["Delta_ECL_MM_USD"].round(2)
df_stress_test["PD_Mean"] = df_stress_test["PD_Mean"].round(4)


# Affichage des résultats
print("\n--- SYNTHÈSE DU STRESS TEST DE TRANSITION (Delta ECL) ---")


# 4.4. Export du livrable final (Phase 7)
out_path = "../reports/tableau_exports/09_transition_stress_test_ecl_summary.csv"
df_stress_test.to_csv(out_path, index=False)
print(f"\nExport final de la Phase 7 (Stress Test) : {out_path} (OK)")



--- SYNTHÈSE DU STRESS TEST DE TRANSITION (Delta ECL) ---

Export final de la Phase 7 (Stress Test) : ../reports/tableau_exports/09_transition_stress_test_ecl_summary.csv (OK)


### Conclusion du Stress Test 

Le Stress Test de Transition révèle que la dégradation des conditions économiques due à une transition mal gérée se traduit directement par une augmentation mesurable des pertes de crédit.

Baseline (Statu Quo) : L'ECL totale est de ≈469 millions USD.

Scénario Désordonné (Choc -10% sur revenu) : L'ECL passe à ≈486 millions USD.

Un scénario de Transition Désordonnée (choc de −10% sur le revenu) 
entraînerait une augmentation de la perte attendue (ECL) de 17.3 millions 
d'USD par rapport au scénario de base. Le pire scénario (Monde Chaud)
entraînerait une perte supplémentaire de 27.9 millions d'USD.